# Sistema Recomendación por género

In [9]:
import sys
sys.path.append("../")
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

from supabase import create_client, Client
from dotenv import load_dotenv
# Cargar variables de entorno
load_dotenv()

# Configuración de Supabase
url = os.getenv("project_url")
key = os.getenv("browser_safe_key")
supabase: Client = create_client(url, key)

In [10]:
def get_brand_genre_ranking(supabase_credential,brand_id):
    brand_ranking = supabase_credential.table("main_genres").select("genre_name","number_of_appearances").eq("brand_id",brand_id).order("number_of_appearances",desc=True).execute().data
    generos = []
    apariciones = []
    for ranking in brand_ranking:
        generos.append(ranking["genre_name"])
        apariciones.append(ranking["number_of_appearances"])

    brand_df = pd.DataFrame({
        "genero" : generos,
        "apariciones" : apariciones
    })

    brand_df.index = brand_df.index + 1
    brand_df.reset_index(inplace=True)
    brand_df.columns = ["ranking","genero","apariciones"]
    brand_df.drop(columns="apariciones",inplace=True)
    brand_df["genero"] = brand_df["genero"].astype(str)
    return brand_df

brand_df = get_brand_genre_ranking(supabase,1)

In [11]:
def get_user_genre_ranking(supabase_credential,user_id):
    user_ranking = supabase_credential.table("user_main_genres").select("genre_name","number_of_appearances").eq("user_id",user_id).order("number_of_appearances",desc=True).execute().data
    generos = []
    apariciones = []
    for ranking in user_ranking:
        generos.append(ranking["genre_name"])
        apariciones.append(ranking["number_of_appearances"])

    user_df = pd.DataFrame({
        "genero" : generos,
        "apariciones" : apariciones
    })

    user_df.index = user_df.index + 1
    user_df.reset_index(inplace=True)
    user_df.columns = ["ranking","genero","apariciones"]
    user_df.drop(columns="apariciones",inplace=True)
    user_df["genero"] = user_df["genero"].astype(str)
    return user_df

user_df = get_user_genre_ranking(supabase,"gonzaloruiperez")


# 1 Buscamos los artistas comunes
Dejando unicamente aquellos que coinciden con su posición en cada ranking!

In [ ]:
def obtener_afinidad_por_genero(supabase_credential,brand_id,user_id):
    # Obtener los rankings de la marca y el usuario
    brand_df = get_brand_genre_ranking(supabase_credential, brand_id)
    user_df = get_user_genre_ranking(supabase_credential, user_id)
    
    # Obtener los generos en común
    generos_comunes = set(user_df['genero']).intersection(set(brand_df['genero']))
    
    # Si no hay generos en común la afinidad es del 0%
    if len(generos_comunes) == 0:
        return 0.0  
    
    user_df_filtrado = user_df[user_df['genero'].isin(generos_comunes)]
    brand_df_filtrado = brand_df[brand_df['genero'].isin(generos_comunes)]

    # Reseteamos el Index
    user_df_filtrado.reset_index(drop=True, inplace=True)
    brand_df_filtrado.reset_index(drop=True, inplace=True)

    # Asignar pesos en base a la posición en el ranking
    user_df_filtrado["peso"] = 1 / user_df_filtrado["ranking"]
    brand_df_filtrado["peso"] = 1 / brand_df_filtrado["ranking"]

    # Normalizar los pesos para evitar sesgos por diferencias de tamaño
    user_df_filtrado["peso"] /= user_df_filtrado["peso"].sum()
    brand_df_filtrado["peso"] /= brand_df_filtrado["peso"].sum()

    # Poner como index genero y peso se queda como columna
    pesos_user = user_df_filtrado.set_index("genero")["peso"]
    pesos_brand = brand_df_filtrado.set_index("genero")["peso"]

    # Ordenamos los generos por orden alfabético para que los vectores estén alineados
    sorted_pesos = sorted(set(pesos_user.index).intersection(set(pesos_brand.index)))

    # Si solo hay un genero en común, asumimos 100% de afinidad
    if len(sorted_pesos) == 1:
        return 100.0 

    # Generamos los vectores
    user_vector = np.array([pesos_user.get(a, 0) for a in sorted_pesos])
    brand_vector = np.array([pesos_brand.get(a, 0) for a in sorted_pesos])

    # Crear matriz de comparación
    matriz_pesos = np.vstack([user_vector, brand_vector])

    # Calcular la matriz de distancias con pdist
    matriz_distancias = squareform(pdist(matriz_pesos, metric="euclidean"))

    # Extraer la distancia entre usuario y marca
    distancia = matriz_distancias[0, 1]

    # Convertimos la distancia en afinidad (invirtiendo la escala)
    afinidad = max(0, (1 - distancia) * 100)

    return float(round(afinidad,2))